To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth?tab=readme-ov-file#-installation-instructions).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

[NEW] Llama-3.1 8b, 70b & 405b are trained on a crazy 15 trillion tokens with 128K long context lengths!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [2]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
* [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [6]:
from Grade2BrailleConverter import Grade2BrailleConverter

In [7]:
g2b = Grade2BrailleConverter()

In [8]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    # for i in range(len(outputs)):
    #     outputs[i] = g2b.to_braille(outputs[i])

    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [7]:
dataset['output']

['⠼⠁⠲⠀⠠⠑⠁⠞⠀⠁⠀⠃⠁⠇⠁⠝⠉⠫⠀⠯⠀⠝⠥⠞⠗⠊⠞⠊⠳⠎⠀⠙⠊⠑⠞⠒⠀⠠⠍⠁⠅⠑⠀⠎⠥⠗⠑⠀⠽⠳⠗⠀⠍⠑⠁⠇⠎⠀⠜⠑⠀⠊⠝⠉⠇⠥⠎⠊⠧⠑⠀⠷⠀⠁⠀⠧⠜⠊⠑⠞⠽⠀⠷⠀⠋⠗⠥⠊⠞⠎⠀⠯⠀⠧⠑⠛⠑⠞⠁⠃⠇⠑⠎⠂⠀⠇⠑⠁⠝⠀⠏⠗⠕⠞⠑⠊⠝⠂⠀⠱⠕⠇⠑⠀⠛⠗⠁⠊⠝⠎⠂⠀⠯⠀⠓⠑⠁⠇⠹⠽⠀⠋⠁⠞⠎⠲⠀⠠⠹⠊⠎⠀⠓⠑⠇⠏⠎⠀⠞⠕⠀⠏⠗⠕⠧⠊⠙⠑⠀⠽⠳⠗⠀⠃⠕⠙⠽⠀⠾⠀⠮⠀⠑⠎⠎⠑⠝⠞⠊⠁⠇⠀⠝⠥⠞⠗⠊⠑⠝⠞⠎⠀⠞⠕⠀⠋⠥⠝⠉⠰⠝⠀⠁⠞⠀⠊⠞⠎⠀⠃⠑⠎⠞⠀⠯⠀⠉⠀⠓⠑⠇⠏⠀⠏⠗⠑⠧⠑⠝⠞⠀⠡⠗⠕⠝⠊⠉⠀⠙⠊⠎⠑⠁⠎⠑⠎⠲\n\n⠼⠃⠲⠀⠠⠑⠝⠛⠁⠛⠑⠀⠔⠀⠗⠑⠛⠥⠇⠜⠀⠏⠓⠽⠎⠊⠉⠁⠇⠀⠁⠉⠞⠊⠧⠊⠞⠽⠒⠀⠠⠑⠭⠻⠉⠊⠎⠑⠀⠊⠎⠀⠉⠗⠥⠉⠊⠁⠇⠀⠿⠀⠍⠁⠊⠝⠞⠁⠊⠝⠬⠀⠎⠞⠗⠕⠝⠛⠀⠃⠕⠝⠑⠎⠂⠀⠍⠥⠎⠉⠇⠑⠎⠂⠀⠯⠀⠉⠜⠙⠊⠕⠧⠁⠎⠉⠥⠇⠜⠀⠓⠑⠁⠇⠹⠲⠀⠠⠁⠊⠍⠀⠿⠀⠁⠞⠀⠇⠑⠁⠎⠞⠀⠼⠁⠑⠚⠀⠍⠊⠝⠥⠞⠑⠎⠀⠷⠀⠍⠕⠙⠻⠁⠞⠑⠀⠁⠻⠕⠃⠊⠉⠀⠑⠭⠻⠉⠊⠎⠑⠀⠕⠗⠀⠼⠛⠑⠀⠍⠊⠝⠥⠞⠑⠎⠀⠷⠀⠧⠊⠛⠕⠗⠳⠎⠀⠑⠭⠻⠉⠊⠎⠑⠀⠑⠁⠡⠀⠺⠑⠑⠅⠲\n\n⠼⠉⠲⠀⠠⠛⠑⠞⠀⠑⠝⠳⠣⠀⠎⠇⠑⠑⠏⠒⠀⠠⠛⠑⠞⠞⠬⠀⠑⠝⠳⠣⠀⠟⠥⠁⠇⠊⠞⠽⠀⠎⠇⠑⠑⠏⠀⠊⠎⠀⠉⠗⠥⠉⠊⠁⠇⠀⠿⠀⠏⠓⠽⠎⠊⠉⠁⠇⠀⠯⠀⠰⠞⠁⠇⠀⠺⠑⠇⠇⠤⠃⠑⠬⠲⠀⠠⠭⠀⠓⠑⠇⠏⠎⠀⠞⠕⠀⠗⠑⠛⠥⠇⠁⠞⠑⠀⠍⠕⠕⠙⠂⠀⠊⠍⠏⠗⠕⠧⠑⠀⠉⠕⠛⠝⠊⠞⠊⠧⠑⠀⠋⠥⠝⠉⠰⠝⠂⠀⠯⠀⠎⠥⠏⠏⠕⠗⠞⠎⠀⠓⠑⠁⠇⠹⠽⠀⠛⠗⠪⠹⠀⠯⠀⠊⠍⠍⠥⠝⠑⠀⠋⠥⠝⠉⠰⠝⠲⠀⠠⠁⠊⠍⠀⠿⠀⠼⠛⠤⠼⠊⠀⠓⠳⠗⠎⠀⠷⠀⠎⠇⠑⠑⠏⠀⠑⠁⠡⠀⠝⠊⠣⠞⠲',
 '⠠⠮⠀⠹⠗⠑⠑⠀⠏⠗⠊⠍⠜⠽⠀⠉⠕⠇⠕⠗⠎⠀⠜⠑⠀⠗⠫⠂⠀⠃⠇⠥⠑⠂⠀⠯⠀⠽⠑⠇⠇⠪⠲⠀⠠⠹⠑⠎⠑⠀⠉⠕⠇⠕⠗⠎⠀⠜⠑⠀⠉⠁⠇⠇⠫⠀⠏⠗⠊⠍⠜⠽⠀⠃⠑⠉⠁⠥⠎⠑⠀⠹⠑⠽⠀⠉⠁⠝⠝⠕⠞⠀⠃⠑⠀⠉⠗⠑⠁⠞⠫⠀⠃⠽⠀⠍⠊⠭⠬⠀⠕⠹⠻⠀⠉⠕⠇⠕⠗⠎⠀⠯⠀⠁⠇⠇⠀⠕⠹⠻⠀⠉⠕⠇⠕⠗⠎⠀⠉⠀⠃⠑⠀⠍⠁⠙⠑⠀⠃⠽⠀⠉⠕⠍⠃⠊⠝⠬⠀⠹⠑⠍⠀⠔⠀⠧⠜⠊⠳⠎⠀⠏⠗⠕⠏⠕⠗⠰⠝⠎⠲⠀⠠⠔⠀⠮⠀⠁⠙⠙⠊⠞⠊⠧⠑⠀⠉⠕⠇⠕⠗⠀⠎⠽⠎⠞⠑⠍⠂⠀⠥⠎⠫⠀⠿⠀⠇⠊⠣⠞⠂⠀⠮⠀⠏⠗⠊⠍⠜⠽⠀⠉⠕⠇⠕⠗⠎⠀⠜⠑⠀⠗⠫⠂⠀⠛⠗⠑⠑⠝⠂⠀⠯⠀⠃⠇⠥⠑⠀⠐⠣⠠⠗⠠⠛⠠⠃⠐⠜⠲',
 '⠠⠁⠝⠀⠁⠞⠕⠍⠀⠊⠎⠀⠮⠀⠃⠁⠎⠊⠉⠀⠃⠥⠊⠇⠙⠬

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 10, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [10]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.586800
2,2.115200
3,1.672900
4,1.863000
5,1.678900
6,1.490100
7,1.080900
8,1.270000
9,1.144400
10,1.122400


In [12]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

496.728 seconds used for training.
8.28 minutes used for training.
Peak reserved memory = 7.922 GB.
Peak reserved memory for training = 1.938 GB.
Peak reserved memory % of max memory = 53.716 %.
Peak reserved memory for training % of max memory = 13.141 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [23]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence and only output in braille.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
response = tokenizer.batch_decode(outputs)[0].split("### Response:")
for i in response[:1]:
    print(i)
print("### Response:")
print(g2b.to_braille(response[1]))

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence and only output in braille.

### Input:
1, 1, 2, 3, 5, 8


### Response:

⠼⠁⠂⠀⠼⠁⠂⠀⠼⠃⠂⠀⠼⠉⠂⠀⠼⠑⠂⠀⠼⠓⠂⠀⠼⠁⠉⠂⠀⠼⠃⠁⠂⠀⠼⠉⠙⠂⠀⠼⠑⠑⠂⠀⠼⠓⠊⠂⠀⠼⠁⠙⠙⠂⠀⠼⠃⠉⠉⠂⠀⠼⠉⠛⠛⠂⠀⠼⠋⠁⠚⠂⠀⠼⠊⠓⠛⠂⠀⠼⠁⠑⠊⠛⠂⠀⠼⠃⠑⠓⠙⠂⠀⠼⠙⠁⠓⠁⠂⠀⠼⠋⠛⠋⠑⠂⠀⠼⠁⠚⠊⠙⠋⠂⠀⠼⠁⠛⠛⠁⠁⠂⠀⠼⠃⠓⠋⠑⠛⠂⠀⠼⠙⠋⠉⠋⠓⠂⠀⠼⠛⠑⠚⠃⠑⠂⠀⠼⠁⠃⠁⠉⠊⠉⠂⠀⠼⠁⠊⠋⠙⠁⠓⠂⠀⠼⠉⠁⠛⠓⠁⠁⠂⠀⠼⠑⠁⠙⠃⠃⠊⠂⠀⠼⠓⠉⠃⠚⠙⠚⠂⠀⠼⠁⠉⠙⠋⠃⠋⠊⠂⠀⠼⠃⠁⠛⠓⠉⠚⠊⠂⠀⠼⠉⠑⠃⠙⠑⠛⠓⠂⠀⠼⠑⠛⠚⠃⠓⠓⠛⠂⠀⠼⠊⠃⠃⠛⠙⠋⠑⠂


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [25]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
response = tokenizer.batch_decode(outputs)[0].split("### Response:")
for i in response[:1]:
    print(i)
print("### Response:")
print(g2b.to_braille(response[1]))

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:



### Response:

⠠⠹⠑⠀⠠⠑⠊⠋⠋⠑⠇⠀⠠⠞⠪⠻⠀⠊⠎⠀⠕⠝⠑⠀⠷⠀⠮⠀⠍⠕⠎⠞⠀⠋⠁⠍⠳⠎⠀⠯⠀⠊⠉⠕⠝⠊⠉⠀⠇⠁⠝⠙⠍⠜⠅⠎⠀⠔⠀⠠⠏⠜⠊⠎⠲⠀⠠⠎⠞⠁⠝⠙⠬⠀⠁⠞⠀⠁⠀⠓⠑⠊⠣⠞⠀⠷⠀⠼⠉⠃⠙⠀⠍⠑⠞⠻⠎⠀⠐⠣⠼⠁⠂⠼⠚⠋⠉⠀⠋⠑⠑⠞⠐⠜⠂⠀⠭⠀⠊⠎⠀⠮⠀⠞⠁⠇⠇⠑⠎⠞⠀⠎⠞⠗⠥⠉⠞⠥⠗⠑⠀⠔⠀⠠⠏⠜⠊⠎⠀⠯⠀⠮⠀⠎⠑⠉⠕⠝⠙⠀⠍⠕⠎⠞⠀⠧⠊⠎⠊⠞⠫⠀⠏⠁⠊⠙⠀⠍⠕⠝⠥⠰⠞⠀⠔⠀⠮⠀⠺⠕⠗⠇⠙⠲⠀⠠⠮⠀⠞⠪⠻⠀⠴⠀⠃⠥⠊⠇⠞⠀⠔⠀⠼⠁⠓⠓⠊⠀⠵⠀⠮⠀⠉⠑⠝⠞⠻⠏⠊⠑⠉⠑⠀⠷⠀⠮⠀⠠⠺⠕⠗⠇⠙⠄⠎⠀⠠⠋⠁⠊⠗⠀⠯⠀⠓⠁⠎⠀⠎⠊⠝⠉⠑⠀⠃⠑⠉⠕⠍⠑⠀⠁⠀⠎⠽⠍⠃⠕⠇⠀⠷⠀⠠⠏⠜⠊⠎⠀⠯⠀⠠⠋⠗⠁⠝⠉⠑⠲⠀⠠⠭⠀⠊⠎⠀⠍⠁⠙⠑⠀⠥⠏⠀⠷⠀⠹⠗⠑⠑⠀⠇⠑⠧⠑⠇⠎⠂⠀⠾⠀⠮⠀⠋⠊⠗⠎⠞⠀⠯⠀⠎⠑⠉⠕⠝⠙⠀⠇⠑⠧⠑⠇⠎⠀⠉⠕⠝⠞⠁⠊⠝⠬⠀⠗⠑⠎⠞⠁⠥⠗⠁⠝⠞⠎⠀⠯⠀⠩⠕⠏⠎⠂⠀⠯⠀⠮⠀⠞⠕⠏⠀⠇⠑⠧⠑⠇⠀⠕⠋⠋⠻⠬⠀⠃⠗⠑⠁⠹⠞⠁⠅⠬⠀⠧⠊⠑⠺⠎⠀⠷⠀⠮⠀⠉⠊⠞⠽⠲⠀⠠⠮⠀⠞⠪⠻⠀⠊⠎⠀⠊⠇⠇⠥⠍⠊⠝⠁⠞⠫⠀⠁⠞⠀⠝⠊⠣⠞⠂⠀⠾⠀⠮⠀⠇⠊⠣⠞⠎⠀⠡⠁⠝⠛⠬⠀⠉⠕⠇⠕⠗⠎⠀⠔⠀⠁⠀⠙⠊⠎⠏⠇⠁⠽⠀⠅⠝⠪⠝⠀⠵⠀⠮⠀⠦⠠⠑⠊⠋⠋⠑⠇
